In [101]:
import numpy as np
import pandas as pd
import json
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

In [132]:
# Carregar o arquivo JSON
with open('/Users/user/Mestrado/K-ISOMAP/main-results/dataset_results_1st_2nd_battery.json', 'r') as file:
    results_1 = json.load(file)

battery_1 = results_1

# Carregar o arquivo JSON
with open('/Users/user/Mestrado/K-ISOMAP/main-results/dataset_results_1st_2nd_battery.json', 'r') as file:
    results_2 = json.load(file)

battery_2 = results_2

# Carregar o arquivo JSON
with open('/Users/user/Mestrado/K-ISOMAP/experiments/dataset_results_3rd_battery.json', 'r') as file:
    results_3 = json.load(file)
    
battery_3 = results_3

# 1st Battery

In [119]:
df = pd.DataFrame(data, columns=columns)
df['Dataset'].unique()

array(['hayes-roth', 'MNIST_norm', 'diggle_table_a2',
       'visualizing_environmental_norm', 'har', 'xd6', 'PhishingWebsites',
       'leukemia', 'cmc_norm', 'AP_Ovary_Lung', 'steel-plates-fault_norm',
       'qsar-biodeg_norm', 'OVA_Uterus_norm', 'steel-plates-fault',
       'newton_hema', 'fri_c4_250_100', 'rabe_131', 'breast-tissue_norm',
       'AP_Endometrium_Breast', 'rabe_131_norm',
       'AP_Endometrium_Breast_norm', 'breast-tissue', 'heart-h_norm',
       'servo', 'diggle_table_a2_norm', 'newton_hema_norm', 'led24',
       'heart-statlog_norm', 'tic-tac-toe_norm', 'eating_norm',
       'spambase_norm', 'OVA_Uterus', 'xd6_norm', 'car-evaluation', 'cmc',
       'har_norm', 'car-evaluation_norm', 'wisconsin_norm', 'tic-tac-toe',
       'prnn_synth', 'conference_attendance', 'AP_Ovary_Lung_norm',
       'heart-h', 'eating', 'micro-mass', 'PhishingWebsites_norm',
       'conference_attendance_norm', 'prnn_synth_norm', 'oh5.wc',
       'visualizing_environmental', 'hayes-roth_nor

In [120]:
# Ordem das métricas
metrics = ["rand", "ca", "fm", "v", "silhouette", "davies_bouldin"]

# Inicializar uma lista para armazenar os dados do DataFrame
data = []

# Iterar sobre os datasets e métodos
for dataset, methods in battery_1.items():
    for method, values in methods.items():
        row = [dataset, method] + [value[0] for value in values]
        data.append(row)

# Criar o DataFrame com colunas dinâmicas
columns = ["Dataset", "Method"] + metrics
df = pd.DataFrame(data, columns=columns)

mask = ~df['Dataset'].str.contains('_norm')

df_filtered = df[mask]

# Definindo a lista de termos a serem filtrados
terms_to_include = [
    'hayes-roth', 
    'diggle_table_a2',
    'visualizing_environmental', 
    'xd6', 
    'PhishingWebsites',
    'steel-plates-fault',
    'qsar-biodeg', 
    'newton_hema', 
    'fri_c4_250_100', 
    'rabe_131', 
    'breast-tissue',
    'heart-h',
    'servo', 
    'led24',
    'heart-statlog', 
    'spambase', 
    'car-evaluation', 
    'cmc',
    'wisconsin',
    'tic-tac-toe',
    'prnn_synth', 
    'conference_attendance', 
    'satimage',
    'Engine1'
]

# Criando a regex para a condição
regex_pattern = '|'.join(terms_to_include)

# Definindo a máscara para filtrar datasets indesejados
mask = df_filtered['Dataset'].str.contains(regex_pattern, case=True)


df_filtered = df_filtered[mask]
df = df_filtered

df['Dataset'].unique()

array(['hayes-roth', 'diggle_table_a2', 'xd6', 'PhishingWebsites',
       'steel-plates-fault', 'newton_hema', 'fri_c4_250_100', 'rabe_131',
       'breast-tissue', 'servo', 'led24', 'car-evaluation', 'cmc',
       'tic-tac-toe', 'prnn_synth', 'conference_attendance', 'heart-h',
       'visualizing_environmental', 'heart-statlog', 'satimage',
       'Engine1', 'qsar-biodeg', 'spambase', 'wisconsin'], dtype=object)

In [121]:
# Definindo os métodos e as colunas
methods = ['KISOMAP', 'ISOMAP', 'UMAP', 'Kernel PCA','t-SNE','Laplacian Eigenmaps','RAW']
statistics = []
data = []
# Iterando sobre todas as combinações de métricas e métodos
for metric in metrics:
    if '_norm' not in df['Dataset'].unique():
        kiso_data = df[df['Method'] == 'KISOMAP'][metric]
        raw_data = df[df['Method'] == 'RAW'][metric]
    for method in methods:
        if method != 'KISOMAP' and method != "RAW":  # Comparar com KISOMAP
            other_data = df[df['Method'] == method][metric]
            if len(kiso_data) == len(other_data):  # Certificar-se de que temos o mesmo número de amostras
                # Realizando os testes
                stat, p_value = friedmanchisquare(raw_data, kiso_data, other_data)
                p_values_nemenyi = sp.posthoc_nemenyi_friedman(np.array([raw_data, kiso_data, other_data]).T)
                
                # Armazenando os resultados
                statistics.append({
                    'Metric': metric,
                    'Method': method,
                    'Stat': stat,
                    'P-Value Friedman': p_value,
                    # Esse aqui é o valor do teste do KISOMAP vs. Other Method
                    'P-Value Nemenyi': p_values_nemenyi[0][2]
                })
                data.append([raw_data.tolist(), kiso_data.tolist(), other_data.tolist()])
            else:
                print(f'Número de amostras não corresponde entre KISOMAP e {method} para a métrica {metric}')

results_df = pd.DataFrame(statistics)

results_df['Test Friedman'] = results_df['P-Value Friedman'].apply(lambda x: 1 if x < 0.05 else 0)
results_df['Test Nemenyi'] = results_df['P-Value Nemenyi'].apply(lambda x: 1 if x < 0.05 else 0)
results_df['P-Value Friedman'] = results_df['P-Value Friedman'].apply(lambda x: f"{x:.3f}")
results_df['P-Value Nemenyi'] = results_df['P-Value Nemenyi'].apply(lambda x: f"{x:.3f}")

results_df

,Metric,Method,Stat,P-Value Friedman,P-Value Nemenyi,Test Friedman,Test Nemenyi
0,rand,ISOMAP,31.600000,0.000,0.566,1,0
1,rand,UMAP,23.083333,0.000,0.731,1,0
2,rand,Kernel PCA,20.583333,0.000,0.813,1,0
3,rand,t-SNE,23.083333,0.000,0.731,1,0
4,rand,Laplacian Eigenmaps,29.250000,0.000,0.397,1,0
5,ca,ISOMAP,46.083333,0.000,0.001,1,1
6,ca,UMAP,33.250000,0.000,0.001,1,1
7,ca,Kernel PCA,39.083333,0.000,0.001,1,1
8,ca,t-SNE,36.750000,0.000,0.001,1,1
9,ca,Laplacian Eigenmaps,34.333333,0.000,0.001,1,1


# 2nd Battery

In [123]:
# Ordem das métricas
metrics = ["rand", "ca", "fm", "v", "silhouette", "davies_bouldin"]

# Inicializar uma lista para armazenar os dados do DataFrame
data = []

# Iterar sobre os datasets e métodos
for dataset, methods in battery_2.items():
    for method, values in methods.items():
        row = [dataset, method] + [value[0] for value in values]
        data.append(row)

# Criar o DataFrame com colunas dinâmicas
columns = ["Dataset", "Method"] + metrics
df = pd.DataFrame(data, columns=columns)


mask = ~df['Dataset'].str.contains('_norm')

df_filtered = df[mask]

# Definindo a lista de termos a serem filtrados
terms_to_include = [
       'cnae-9',
       'leukemia','AP_Ovary_Lung',
       'OVA_Uterus', 
       'AP_Endometrium_Breast', 
       'eating',
       'AP_Ovary_Lung',
       'micro-mass',
       'har',
       'oh5.wc',
       'AP_Breast_Kidney']

# Criando a regex para a condição
regex_pattern = '|'.join(terms_to_include)

# Definindo a máscara para filtrar datasets indesejados
mask = df_filtered['Dataset'].str.contains(regex_pattern, case=True)

df_filtered = df_filtered[mask]
df = df_filtered

df['Dataset'].unique()

array(['har', 'leukemia', 'AP_Ovary_Lung', 'AP_Endometrium_Breast',
       'OVA_Uterus', 'eating', 'micro-mass', 'oh5.wc', 'AP_Breast_Kidney',
       'cnae-9'], dtype=object)

In [124]:
# Definindo os métodos e as colunas
methods = ['KISOMAP', 'ISOMAP', 'UMAP', 'Kernel PCA','t-SNE','Laplacian Eigenmaps','RAW']
statistics = []
data = []
# Iterando sobre todas as combinações de métricas e métodos
for metric in metrics:
    if '_norm' not in df['Dataset'].unique():
        kiso_data = df[df['Method'] == 'KISOMAP'][metric]
        raw_data = df[df['Method'] == 'RAW'][metric]
    for method in methods:
        if method != 'KISOMAP' and method != "RAW":  # Comparar com KISOMAP
            other_data = df[df['Method'] == method][metric]
            if len(kiso_data) == len(other_data):  # Certificar-se de que temos o mesmo número de amostras
                # Realizando os testes
                stat, p_value = friedmanchisquare(raw_data, kiso_data, other_data)
                p_values_nemenyi = sp.posthoc_nemenyi_friedman(np.array([raw_data, kiso_data, other_data]).T)
                
                # Armazenando os resultados
                statistics.append({
                    'Metric': metric,
                    'Method': method,
                    'Stat': stat,
                    'P-Value Friedman': p_value,
                    # Esse aqui é o valor do teste do KISOMAP vs. Other Method
                    'P-Value Nemenyi': p_values_nemenyi[0][2]
                })
                data.append([raw_data.tolist(), kiso_data.tolist(), other_data.tolist()])
            else:
                print(f'Número de amostras não corresponde entre KISOMAP e {method} para a métrica {metric}')

results_df = pd.DataFrame(statistics)

results_df['Test Friedman'] = results_df['P-Value Friedman'].apply(lambda x: 1 if x < 0.05 else 0)
results_df['Test Nemenyi'] = results_df['P-Value Nemenyi'].apply(lambda x: 1 if x < 0.05 else 0)
results_df['P-Value Friedman'] = results_df['P-Value Friedman'].apply(lambda x: f"{x:.3f}")
results_df['P-Value Nemenyi'] = results_df['P-Value Nemenyi'].apply(lambda x: f"{x:.3f}")

results_df

,Metric,Method,Stat,P-Value Friedman,P-Value Nemenyi,Test Friedman,Test Nemenyi
0,rand,ISOMAP,16.800000,0.000,0.373,1,0
1,rand,UMAP,12.200000,0.002,0.010,1,1
2,rand,Kernel PCA,9.600000,0.008,0.900,1,0
3,rand,t-SNE,12.600000,0.002,0.002,1,1
4,rand,Laplacian Eigenmaps,8.051282,0.018,0.437,1,0
5,ca,ISOMAP,20.000000,0.000,0.065,1,0
6,ca,UMAP,15.800000,0.000,0.001,1,1
7,ca,Kernel PCA,15.200000,0.001,0.001,1,1
8,ca,t-SNE,15.800000,0.000,0.010,1,1
9,ca,Laplacian Eigenmaps,15.200000,0.001,0.005,1,1


# 3rd Battery

In [128]:
# Ordem das métricas
metrics = ["rand", "ca", "fm", "v", "silhouette", "davies_bouldin"]

# Inicializar uma lista para armazenar os dados do DataFrame
data = []

# Iterar sobre os datasets e métodos
for dataset, methods in battery_3.items():
    for method, values in methods.items():
        row = [dataset, method] + [value[0] for value in values]
        data.append(row)

# Criar o DataFrame com colunas dinâmicas
columns = ["Dataset", "Method"] + metrics
df = pd.DataFrame(data, columns=columns)

mask = ~df['Dataset'].str.contains('_norm')

df_filtered = df[mask]

# Definindo a lista de termos a serem filtrados
terms_to_include = [
       'MNIST',
       'F-MNIST',
       'olivetti-faces',
       'blood-transfusion',
       'Pen-Digits',
       'coil-20'
       ]

# Criando a regex para a condição
regex_pattern = '|'.join(terms_to_include)

# Definindo a máscara para filtrar datasets indesejados
mask = df_filtered['Dataset'].str.contains(regex_pattern, case=True)

df_filtered = df_filtered[mask]
df = df_filtered

df['Dataset'].unique()

array(['MNIST', 'olivetti-faces', 'F-MNIST', 'Pen-Digits',
       'blood-transfusion', 'coil-20'], dtype=object)

In [129]:
# Definindo os métodos e as colunas
methods = ['KISOMAP', 'ISOMAP', 'UMAP', 'Kernel PCA','t-SNE','Laplacian Eigenmaps','RAW']
statistics = []
data = []
# Iterando sobre todas as combinações de métricas e métodos
for metric in metrics:
    if '_norm' not in df['Dataset'].unique():
        kiso_data = df[df['Method'] == 'KISOMAP'][metric]
        raw_data = df[df['Method'] == 'RAW'][metric]
    for method in methods:
        if method != 'KISOMAP' and method != "RAW":  # Comparar com KISOMAP
            other_data = df[df['Method'] == method][metric]
            if len(kiso_data) == len(other_data):  # Certificar-se de que temos o mesmo número de amostras
                # Realizando os testes
                stat, p_value = friedmanchisquare(raw_data, kiso_data, other_data)
                p_values_nemenyi = sp.posthoc_nemenyi_friedman(np.array([raw_data, kiso_data, other_data]).T)
                
                # Armazenando os resultados
                statistics.append({
                    'Metric': metric,
                    'Method': method,
                    'Stat': stat,
                    'P-Value Friedman': p_value,
                    # Esse aqui é o valor do teste do KISOMAP vs. Other Method
                    'P-Value Nemenyi': p_values_nemenyi[0][2]
                })
                data.append([raw_data.tolist(), kiso_data.tolist(), other_data.tolist()])
            else:
                print(f'Número de amostras não corresponde entre KISOMAP e {method} para a métrica {metric}')

results_df = pd.DataFrame(statistics)

results_df['Test Friedman'] = results_df['P-Value Friedman'].apply(lambda x: 1 if x < 0.05 else 0)
results_df['Test Nemenyi'] = results_df['P-Value Nemenyi'].apply(lambda x: 1 if x < 0.05 else 0)
results_df['P-Value Friedman'] = results_df['P-Value Friedman'].apply(lambda x: f"{x:.3f}")
results_df['P-Value Nemenyi'] = results_df['P-Value Nemenyi'].apply(lambda x: f"{x:.3f}")

results_df

,Metric,Method,Stat,P-Value Friedman,P-Value Nemenyi,Test Friedman,Test Nemenyi
0,rand,ISOMAP,2.333333,0.311,0.319,0,0
1,rand,UMAP,6.333333,0.042,0.107,1,0
2,rand,Kernel PCA,7.000000,0.030,0.025,1,1
3,rand,t-SNE,6.333333,0.042,0.107,1,0
4,rand,Laplacian Eigenmaps,9.333333,0.009,0.011,1,1
5,ca,ISOMAP,12.000000,0.002,0.193,1,0
6,ca,UMAP,10.333333,0.006,0.004,1,1
7,ca,Kernel PCA,9.000000,0.011,0.025,1,1
8,ca,t-SNE,9.333333,0.009,0.055,1,0
9,ca,Laplacian Eigenmaps,4.333333,0.115,0.319,0,0


In [130]:
# First set of experiments
    ##{"db": skdata.fetch_openml(name='servo', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='car-evaluation', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='breast-tissue', version=2), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='Engine1', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='xd6', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='heart-h', version=3), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='steel-plates-fault', version=3), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0}]
    ##{"db": skdata.fetch_openml(name='PhishingWebsites', version=1), "reduce_samples": True, "percentage":.1, "reduce_dim":False, "num_features": 0},              # 10% of the samples
    ##{"db": skdata.fetch_openml(name='satimage', version=1), "reduce_samples": True, "percentage":.25, "reduce_dim":False, "num_features": 0},                     # 25% of the samples
    ##{"db": skdata.fetch_openml(name='led24', version=1), "reduce_samples": True, "percentage":.20, "reduce_dim":False, "num_features": 0},                        # 20% of the samples
    ##{"db": skdata.fetch_openml(name='hayes-roth', version=2), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='rabe_131', version=2), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='prnn_synth', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='visualizing_environmental', version=2), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='diggle_table_a2', version=2), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='newton_hema', version=2), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='wisconsin', version=2), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='fri_c4_250_100', version=2), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='conference_attendance', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='tic-tac-toe', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='qsar-biodeg', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='spambase', version=1), "reduce_samples": True, "percentage":.25, "reduce_dim":False, "num_features": 0},                     # 25% of the samples
    ##{"db": skdata.fetch_openml(name='cmc', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0},
    ##{"db": skdata.fetch_openml(name='heart-statlog', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features": 0}]
    
    #Second set of experiments
    ##{"db": skdata.fetch_openml(name='cnae-9', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":True, "num_features": 50},                     # 50-D
    ##{"db": skdata.fetch_openml(name='AP_Breast_Kidney', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":True, "num_features": 500},          # 500-D
    ##{"db": skdata.fetch_openml(name='AP_Endometrium_Breast', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":True, "num_features": 400},     # 400-D
    ##{"db": skdata.fetch_openml(name='AP_Ovary_Lung', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":True, "num_features": 100},             # 100-D
    ##{"db": skdata.fetch_openml(name='OVA_Uterus', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":True, "num_features": 100},                # 100-D
    ##{"db": skdata.fetch_openml(name='micro-mass', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":True, "num_features": 100},                # 100-D
    ##{"db": skdata.fetch_openml(name='har', version=1), "reduce_samples": True, "percentage":0.1, "reduce_dim":True, "num_features": 100},                      # 10%  of the samples and 100-D
    ##{"db": skdata.fetch_openml(name='eating', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":True, "num_features": 100},                    # 100-D
    ##{"db": skdata.fetch_openml(name='oh5.wc', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":True, "num_features": 40},                     # 40-D
    ##{"db": skdata.fetch_openml(name='leukemia', version=1), "reduce_samples": False, "percentage":0, "reduce_dim":True, "num_features": 40}]                 # 40-D

    #Third set of experiments
    #{"db": blood_transfusion, "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features":0}]
    #{"db": olivetti, "reduce_samples": False, "percentage":0, "reduce_dim":True, "num_features":100}]
    #{"db": coil_20, "reduce_samples": False, "percentage":0.1, "reduce_dim":True, "num_features":100}]
    #{"db": pen_digits, "reduce_samples": False, "percentage":0, "reduce_dim":False, "num_features":0}]
    #{"db": mnist, "reduce_samples": True, "percentage":0.10, "reduce_dim":True, "num_features":200}]
    #{"db": fmnist, "reduce_samples": True, "percentage":0.10, "reduce_dim":True, "num_features":200}]